## Diffusion Weighted Imaging

In this lab, we will be treading some old grounds with a new perspective. 
Functional (fMRI) and diffusion (dMRI) data require similar processing such as motion correction, denoising, masking, and modelling. 
We will complete this lab using only python commands, which means we will see familiar FSL commands in a new light.

**Learning Objectives**
- Learn the steps to process diffusion weighted data
- Be aware of some common gotchas! (e.g. rotating B vectors and data representations)
- Learn Nipype basics


In [ ]:
%%bash
# lab prep
# need access to ants utilities
path='export PATH="/usr/lib/ants:$PATH"'
proper_path=$(grep "${path}" ~/.bashrc)
if [[ ${proper_path} == "" ]]; then
    echo ${path} >> ~/.bashrc
fi

# need another python module
pip install scikit-image

# install dipy
conda install dipy vtk -c conda-forge

if [ ! -d 'derivatives' ]; then
    wget --quiet -O 11-Lab_data.tar.gz https://osf.io/hde8t/download &&\
    tar -xf 11-Lab_data.tar.gz &&\
    rm 11-Lab_data.tar.gz
fi


# exit the notebook, and type source ~/.bashrc in the terminal and open the notebook again.

In [ ]:
# notebook prep
%matplotlib inline
# modules to preprocess the data
from nipype.workflows.dmri.fsl.artifacts import hmc_pipeline
from dwi_preprocess import ecc_pipeline
from nipype.workflows.dmri.fsl.utils import rotate_bvecs

# nipype utilities for building workflows
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio
from nipype.interfaces import utility as niu

# for various file operations
import os

# making a mask
from dipy.segment.mask import median_otsu

# for loading nifti files into python
import nibabel as nib

# for applying mathematical oeprations to nifti objects
import numpy as np

# visualizing results
from nilearn import plotting

## Step 1: Masking
We have seen masking using [bet](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET/UserGuide), but sometimes a simpler method suffices. The [median otsu](https://en.wikipedia.org/wiki/Otsu%27s_method) is another image processing algorithm that's found use in the world of MRI.We will be using [dipy's implementation of this algorithm](http://nipy.org/dipy/reference/dipy.segment.html#dipy.segment.mask.median_otsu)

In [ ]:
# the input file
dwi_nii = './sub-999/ses-activepre/dwi/sub-999_ses-activepre_dwi.nii.gz'

# load the file into python
dwi_img = nib.load(dwi_nii)

# access the 4D data matrix in the python object
dwi_data = dwi_img.get_data()

# mask the data
_, mask_data = median_otsu(dwi_data)

# make the mask a python image object
mask_img = nib.Nifti1Image(mask_data.astype(np.float32), dwi_img.affine)

# set the output directory
fout = os.path.join(os.getcwd(), 
                    'derivatives/otsu_mask/sub-999/ses-activepre/dwi')

# make the output directory
os.makedirs(fout, exist_ok=True)

# set the output name of the file
fname = 'sub-999_ses-activepre_dwi_mask.nii.gz'
out_fname = os.path.join(fout, fname)

# save the output as a nifti file
fmask = nib.save(mask_img, out_fname)

In [ ]:
# visualize the mask
# grab the first volume of the dwi (a B0)
first_vol_data = dwi_data[:,:,:,0]
# make the volume a python nifti object
first_vol_img = nib.Nifti1Image(first_vol_data.astype(np.float32), dwi_img.affine)
# plot the mask
plotting.plot_roi(roi_img=out_fname, bg_img=first_vol_img)

## Step 2: Head Motion/Eddy Current Correction
Just as people move during resting state or task fmri, they will also move when we collect diffusion weighted imaging. In addition, due to [eddy currents](http://mriquestions.com/eddy-current-problems.html), there are shears in the data unique to each magnetic vector (direction) applied. In our example  scan, we have 64 unique directions, and therefor the data are sheared in 64 unique ways. To convince ourselves of this, let's take a look at some data.

In [ ]:
for vol_idx in range(10, 60, 10):
    tmp_img = nib.Nifti1Image(dwi_data[:,:,:,vol_idx], dwi_img.affine)
    plotting.plot_anat(tmp_img)

Below we are going to setup a "[workflow](https://miykael.github.io/nipype_tutorial/notebooks/basic_workflow.html)" using [nipype](http://nipype.readthedocs.io/en/latest/). A workflow is a set of processing steps we would like to run together. For example, FEAT completes multiple steps for slice-timing correction, motion correction, spatial smoothing, temporal filtering, etc. Nipype gives us the ability to contruct these workflows all within python.

### DO NOT RUN THE CODE IN THE BELOW BLOCK (I've ran it for you...it takes 30-45 minutes)

In [ ]:
# make a working directory for intermediate outputs
os.makedirs('work', exist_ok=True)
# set a variable to that working directory
work_dir = os.path.join(os.getcwd(), 'work')

# set the templates for the files we want to match
templates = dict(dwi_nii="sub-{sub_label}/ses-{ses_label}/dwi/sub-{sub_label}_ses-{ses_label}_dwi.nii.gz",
                 bvals="sub-{sub_label}/ses-{ses_label}/dwi/sub-{sub_label}_ses-{ses_label}_dwi.bval",
                 bvecs="sub-{sub_label}/ses-{ses_label}/dwi/sub-{sub_label}_ses-{ses_label}_dwi.bvec",
                 mask="derivatives/otsu_mask/sub-{sub_label}/ses-{ses_label}/dwi/sub-{sub_label}_ses-{ses_label}_dwi_mask.nii.gz")

# make the node to match the templates
inputnode = pe.Node(nio.SelectFiles(templates), name='inputnode')

# set the node inputs to the desired inputs
inputnode.inputs.base_directory = os.getcwd()
inputnode.inputs.sub_label = '999'
inputnode.inputs.ses_label = 'activepre'


# head motion correction workflow
hmc_wf = hmc_pipeline(name='hmc_wf')

# eddy current correction workflow
ecc_wf = ecc_pipeline(name='ecc_wf')

# rotate the transformed b vectors
rotate_bvec = pe.Node(
    niu.Function(
        input_names=['in_bvec', 'in_matrix'],
        output_names=['out_file'],
        function=rotate_bvecs),
        name='rotate_bvec')

# collect the outputs
ds_output = pe.Node(nio.DataSink(), name='ds_output')
ds_output.inputs.base_directory = os.path.join(os.getcwd(), 'derivatives/preproc/sub-999/ses-activepre/dwi/')

# the workflow
dwi_preproc = pe.Workflow(name='dwi_preproc')

dwi_preproc.connect([
    (inputnode, hmc_wf, [('dwi_nii', 'inputnode.in_file'),
                         ('mask', 'inputnode.in_mask'),
                         ('bvals', 'inputnode.in_bval'),
                         ('bvecs', 'inputnode.in_bvec')]),
    (inputnode, ecc_wf, [('bvals', 'inputnode.in_bval'),
                         ('dwi_nii', 'inputnode.in_file'),
                         ('mask', 'inputnode.in_mask')]),
    (hmc_wf, ecc_wf, [('outputnode.out_xfms', 'inputnode.in_xfms')]),
    (ecc_wf, rotate_bvec, [('outputnode.out_xfms', 'in_matrix')]),
    (inputnode, rotate_bvec, [('bvecs', 'in_bvec')]),
    (rotate_bvec, ds_output, [('out_file', 'bvec')]),
    (ecc_wf, ds_output, [('outputnode.out_file', 'dwi_eddy_corr')]),
])

# place all the intermediate results in the work directory
dwi_preproc.base_dir = work_dir
# run the workflow
dwi_preproc.run()
# make an illustration of the workflow
dwi_preproc.write_graph()

![workflow](work/dwi_preproc/graph.png "so workflowy")

## Step 3: Read in the processed diffusion data and denoise with local PCA
Now that we've motion/eddy corrected the image, we will do some additional denoising of the data to boost our signal to noise ratio using [dipy](http://nipy.org/dipy/). Specifically we will be using [local PCA](http://nipy.org/dipy/examples_built/denoise_localpca.html)

In [ ]:
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import fractional_anisotropy
import dipy.reconst.dti as dti
import numpy as np
from dipy.denoise.localpca import localpca
from dipy.denoise.pca_noise_estimate import pca_noise_estimate

In [ ]:
fdwi_preproc = os.path.join(os.getcwd(), 'derivatives/preproc/sub-999/ses-activepre/dwi/dwi_eddy_corr/sub-999_ses-activepre_dwi_eccorrect.nii.gz')
fbval = os.path.join(os.getcwd(), './sub-999/ses-activepre/dwi/sub-999_ses-activepre_dwi.bval')
fbvec_preproc = os.path.join(os.getcwd(), 'derivatives/preproc/sub-999/ses-activepre/dwi/bvec/sub-999_ses-activepre_dwi_rotated.bvec')


In [ ]:
# load the diffusion image as a python object
img_preproc = nib.load(fdwi_preproc)

# access the array of the python object
data_preproc = img_preproc.get_data()

In [ ]:
# mask the preprocessed data (becomes a 2D array)
mask_preproc_data = data_preproc[mask_data]

# create the correctly shaped array (128, 128, 70, 74)
data_preproc_masked = np.zeros(data_preproc.shape, dtype=data_preproc.dtype)

# fill in the 4D array of zeros with data from the 2D array
data_preproc_masked[mask_data] = mask_preproc_data

In [ ]:
# read the bvals and bvecs into python
bvals, bvecs_preproc = read_bvals_bvecs(fbval, fbvec_preproc)
gtab_preproc = gradient_table(bvals, bvecs_preproc)


In [ ]:
# determine the sigma to use for denoising
sigma = pca_noise_estimate(data_preproc_masked, gtab_preproc, correct_bias=True, smooth=3)
sigma.shape

In [ ]:
# denoise data with localpca
data_denoised = localpca(data_preproc_masked, sigma=sigma, patch_radius=2)



## Step 4: Fit a tensor model on our preprocessed diffusion weighted data
We can now fit a tensor model using [dipy](http://nipy.org/dipy/)!

In [ ]:
# make the tensor model with the vectors and fit it to our data
tenmodel_preproc = dti.TensorModel(gtab_preproc)
tenfit_preproc = tenmodel_preproc.fit(data_denoised)

## Step 5: Calculate Fractional Anisotropy
Remember fraction anisotropy is measured using the dominant gradient direction in the numerator and the two orthogonal gradients in the demoninator.
$$FA = \sqrt{3/2} * \frac{\sqrt{(\lambda_1 - \hat{\lambda})^2 + (\lambda_2 - \hat{\lambda})^2 + (\lambda_3 - \hat{\lambda})^2}} {\sqrt{\lambda_1^2 + \lambda_2^2 + \lambda_3^2}}$$

In [ ]:
# calculate fractional anisotropy
fa_preproc = fractional_anisotropy(tenfit_preproc.evals)
# zero any elements that are NaN (Not a Number)
fa_preproc[np.isnan(fa_preproc)] = 0
# Save the fa_output as an image
fa_img = nib.Nifti1Image(fa_preproc.astype(np.float32), img_preproc.affine)
nib.save(fa_img, 'fa_preproc.nii.gz')
fa_file = os.path.join(os.getcwd(), 'fa_preproc.nii.gz')

## Step 6: Measure FA of the Forceps Major

In [ ]:
from nipype.interfaces import fsl

In [ ]:
# FSL's FA template
template = '/usr/share/fsl/data/standard/FMRIB58_FA_1mm.nii.gz'

# setup flirt sub2mni
flirt = fsl.FLIRT(in_file=fa_file, 
                  reference=template, 
                  out_matrix_file='dwi2mni.mat')

# display the commandline
display(flirt.cmdline)

# run the function
flirt_res = flirt.run()

In [ ]:
# setup fnirt sub2mni
fnirt = fsl.FNIRT(in_file=fa_file, 
                  affine_file=flirt_res.outputs.out_matrix_file,
                  ref_file=template,
                  config_file='FA_2_FMRIB58_1mm',
                  field_file='dwi2mni_warp')

# display the commandline
display(fnirt.cmdline)

# run the function
fnirt_res = fnirt.run()

In [ ]:
# take the inverse of the warp generated by fnirt
invwarp = fsl.InvWarp(warp='dwi2mni_warp.nii.gz',
                      reference=fa_file,
                      inverse_warp='mni2dwi_warp.nii.gz')

display(invwarp.cmdline)
invwarp_res = invwarp.run()

In [ ]:
# source file for tracts /usr/share/fsl/data/atlases/JHU-tracts.xml

# this file contains the tracts listed in the above xml file
tract_atlas = '/usr/share/fsl/data/atlases/JHU/JHU-ICBM-tracts-maxprob-thr25-2mm.nii.gz'

# Select the forceps major from the atlas
forceps_major = fsl.Threshold(in_file=tract_atlas,
                              thresh=8,
                              args='-uthr 8 -bin',
                              out_file='forceps_major.nii.gz')

display(forceps_major.cmdline)
forceps_major_res = forceps_major.run()

In [ ]:
plotting.plot_roi(forceps_major_res.outputs.out_file, bg_img=template)

In [ ]:
# apply the mni2dwi_warp to the forceps_major
forceps_major_sub = fsl.ApplyWarp(in_file=forceps_major_res.outputs.out_file, 
                                  ref_file=fa_file,
                                  interp='nn',
                                  field_file=invwarp_res.outputs.inverse_warp,
                                  out_file='sub-999_forceps_major.nii.gz')

display(forceps_major_sub.cmdline)
forceps_major_sub_res = forceps_major_sub.run()

In [ ]:
plotting.plot_roi(forceps_major_sub_res.outputs.out_file, bg_img=fa_file)

In [ ]:
# extract the average FA from the Forceps Major
forceps_major_fa = fsl.ImageStats(in_file=fa_file, 
                                  mask_file=forceps_major_sub_res.outputs.out_file,
                                  op_string='-M')

display(forceps_major_fa.cmdline)
forceps_major_fa_res = forceps_major_fa.run()

In [ ]:
# display the result
forceps_major_fa_res.outputs.out_stat

## Problem Set

1) Let's see how preprocessing has impacted our results. Instead of doing all the preprocessing, let's use the raw unprocessed data instead. 
        - the raw dwi file is here: ./sub-999/ses-activepre/dwi/sub-999_ses-activepre_dwi.nii.gz
        - the raw bvec file is here: ./sub-999/ses-activepre/dwi/sub-999_ses-activepre_dwi.bvec
The output will be the average FA of the Forceps Major (processed forceps_major_FA_ave = 0.243983)

In [ ]:
# code here

2) Choose another tract you are interested in and calculate the average FA of that region.

Outputs:
- Plot the region you chose in both MNI and subject space like we did in lab.
- The FA statistic from the region you chose

In [ ]:
# code here